<a href="https://colab.research.google.com/github/An0816/GAN/blob/main/VanillaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.utils.data as data

In [ ]:
# custom data를 processing하기 위해, Dataset class를 상속받아서, 새로운 Dataset 클래스를 만듦
# __init__, __len__, __getitem 선언
# __init__(self, 매개변수) : 클래스 내에서 사용할 매개변수 정의 
# __len__(self) : 데이터의 총 길이를 리턴
# __getitem__(self, index) : dataset에서 특정 index에 있는 sample을 가져옴
class Dataset(data.Dataset):
    def __init__(self):
        super().__init__()
        self.dataset = datasets.MNIST(root = 'MNIST/', 
                                      train = True, download = True, 
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean = (0.5,), std = (0.5,))
                                                                                                                      
        ]))
    # train = True : create dataset from `MNIST/processed/training.pt`
    # download = True : download the dataset from the internet and put it in root directory
    # transform : 이미지 변형
    # ToTensor() : numpy 배열의 이미지를 tensor 타입으로 변경해줌
    # Normalize(mean, std, inplace = False) : 이미지를 정규화
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

# dataLoader : dataset을 인자로 받아서 data를 뽑아냄
class DataLoader:
    def __init__(self, dataset):
        super(data.DataLoader).__init__()

        self.dataloader = data.DataLoader(dataset = dataset,
                                          batch_size = 1,
                                          shuffle = True,
                                          drop_last = True)
        
# interpreter에서 직접 실행된 경우에만 if문 이하의 코드를 돌려라
if __name__ == "__main__":
    dataset = Dataset()
    dataLoader = DataLoader(dataset)


In [ ]:
class Loss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.loss = torch.nn.BCELoss()
    def forward(self, x, y):
        return self.loss(x, y)

if __name__ == "__main__":
    x = torch.rand(3, 1)
    y = torch.rand(3, 1)
    loss = Loss()

    print(loss(x, y))

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(100, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 784),
            torch.nn.Tanh()
        )
    def forward(self, x):
        x = self.model(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self,x):
        x = self.model(x)
        return x


In [ ]:
dataset = Dataset()
data_loader = DataLoader(dataset)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

G = Generator().to(device)
D = Discriminator().to(device)
criterion = Loss()

optim_G = torch.optim.Adam(G.parameters(), lr = 0.0001)
optim_D = torch.optim.Adam(D.parameters(), lr = 0.0001)

In [ ]:
total_epochs = 200
total_batch = len(data_loader.dataloader)

for epcoh in range(total_epochs):
    avg_cost = [0, 0]
    for x, _ in data_loader.dataloader:
        real = (torch.FloatTensor(x.size(0), 1).fill_(1.0)).to(device)
        fake = (torch.FloatTensor(x.size(0), 1).fill_(0.0)).to(device)

        x = x.view(x.size(0), -1).to(device)

        noise = torch.randn(data_loader.dataloader.batch_size, 100, device = device)

        fake_img = G(noise)

        # train Generator
        optim_G.zero_grad()
        g_cost = criterion(D(fake_img), real)
        g_cost.backward()
        optim_G.step()

        fake_img = fake_img.detach().to(device) # 얘를 굳이 하는 이유가 머야

        # train Discriminator
        optim_D.zero_grad()
        d_cost = criterion(D(torch.cat((x, fake_img))), torch.cat((real, fake)))
        d_cost.backward()
        optim_D.step()

        avg_cost[0] += g_cost
        avg_cost[1] += d_cost

    avg_cost[0] /= total_batch
    avg_cost[1] /= total_batch

    if (epoch+1) % 10 == 0:
        fake_img = fake_img.reshape([batch_size, 1, 28, 28])
        img_grid = make_grid(fake_img, nrow=10, normalize=True)
        save_image(img_grid, "/content/drive/MyDrive/Deep Learning/GAN/GAN Result/%d.png"%(epoch+1))
        print("Epoch: %d, Generator: %f, Discriminator: %f"%(epoch+1, avg_cost[0], avg_cost[1]))

    if (epcoh+1) % 10 == 0:
        fake_img = fake_img.reshape([batch_size, 1, 28, 28])
        img_grid = make_grid(fake_img, nrow = 10, normalize = True)
        save_image(img_grid, "/content/drive/MyDrive/Deep Learning/GAN/result/%d.png"%(epoch + 1))
        print(f"Epoch : {epoch + 1},\t Generator = {avg_cost[0]},\t Discriminator = {avg_cost[1]}")
